# Steps
✅ Get dataset of 20 stock from database (.csv) <br>
⬜ 

# Further ideas
1. Give multiple subsets of stocks (e.g. "Becky" as a strategy)


# Selected stocks
20 stocks were randomly selected from the S&P 500 pool

In [1]:
stock_symbols = ['MCD', 'ESS', 'ABC', 'PFE', 'MA', 'GWW', 'AOS', 'GD', 'YUM', 'LOW', 'ALB', 'IFF', 'TPR', 'CB', 'ANTM', 'WAT', 'CPB', 'IPGP', 'SBUX', 'HSY']

## Imports

In [1]:
# type hinting imports
from typing import Dict
from pandas._libs.tslibs.timestamps import Timestamp

from collections import namedtuple
import pprint
import math
import requests
import os
import pandas as pd
import time
from alpha_vantage.timeseries import TimeSeries

## Getting the data

In [ ]:
full_data = []

for symbol in stock_symbols:
    query_params["symbol"] = symbol
    resp = requests.get(url=url, params=query_params)
    resp.raise_for_status()

In [2]:
API_KEY = "1077F37TLBGNSVY2"

STANDARD_DELAY = 60 // 5
DATA_DIR_PATH = r"C:\Users\Jonas\Desktop\CGI\dev\exp\data"
ts = TimeSeries(key=API_KEY, output_format="pandas")

CSV_COLS = ["symbol", "date", "1. open", "2. high", "3. low", "4. close", "5. adjusted close",
           "6. volume", "7. dividend amount", "8. split coefficient"]

### Writing from av API to .csv files

In [9]:
master_df = pd.DataFrame(columns=CSV_COLS)

for idx, symbol in enumerate(stock_symbols):
#     file_path = os.path.join(DATA_DIR_PATH, f"{symbol}.csv")
    df, _ = ts.get_daily_adjusted(symbol=symbol, outputsize="full")
    df["symbol"] = symbol
    master_df.append(other=df)
    print(f"Appended stock {idx}: <{symbol}>. Waiting...")
    time.sleep(STANDARD_DELAY)

Appended stock 0: <MCD>. Waiting...
Appended stock 1: <ESS>. Waiting...


KeyboardInterrupt: 

### Merging all .csv files

In [6]:
file_names = [os.path.join(DATA_DIR_PATH, f) for f in os.listdir(DATA_DIR_PATH) \
                  if os.path.isfile(os.path.join(DATA_DIR_PATH, f))]

In [7]:
df_list = []
for f in file_names:
    df = pd.read_csv(f, index_col=None, header=0)
    df['symbol'] = f.split("\\")[-1].split(".")[0]
    df_list.append(df)
    
df = pd.concat(df_list, axis=0, ignore_index=True)

### Preprocessing

In [8]:
PRICE_COLUMNS = [
    'date',
    'open',
    'high',
    'low',
    'close',
    'adjusted_close',
    'volume',
    'dividend_amount',
    'split_coefficient',
    'symbol',
]

In [9]:
cols_to_rename = {}
for c in df.columns:
    cols_to_rename[c] = "".join(list(filter(lambda s: s.isalpha() or s == " ", c))).strip()
    cols_to_rename[c] = cols_to_rename[c].replace(" ", "_")
    
df.rename(columns=cols_to_rename, inplace=True)

In [10]:
df['date'] = pd.to_datetime(df['date'])
df['symbol'] = df['symbol'].astype("string") # even though this doesn't do anything
df.dtypes

date                 datetime64[ns]
open                        float64
high                        float64
low                         float64
close                       float64
adjusted_close              float64
volume                      float64
dividend_amount             float64
split_coefficient           float64
symbol                       string
dtype: object

# Algorithm

## Class definitions

In [11]:
class Market:
    def __init__(self, prices: pd.DataFrame, name: str = None):
        assert not prices.empty and all([c in PRICE_COLUMNS for c in prices.columns]), \
            "Wrong DataFrame format!"
        self.prices = prices
        # sort by date ascending, then by symbol descending
        self.prices.sort_values(by=["date", "symbol"], inplace=True, ascending=[True, False])
        self.name = name or "default_market"
        self.max_date = None
    
    @property
    def available_symbols(self):
        return list(self.prices.symbol.unique())
    
    @property
    def current_date(self):
        return self.max_date or "Market has not yet started!"
    
    def get_most_recent_price(self, symbol: str) -> float:
        stock_mask = (self.prices.date == self.max_date) & (self.prices.symbol == symbol)
        return self.prices.loc[stock_mask].adjusted_close.values[0]
        
    def pay_dividends(self, stock_portfolio: Dict[str, float]) -> float:
        if not stock_portfolio:
            return 0.
        dividend_amount = 0
        for stock, amount in stock_portfolio.items():
            dividend_mask = (self.prices.date == self.max_date) & (self.prices.symbol == stock)
            dividend_amount += self.prices.loc[dividend_mask].dividend_amount.sum() * amount
        if dividend_amount:
            print(f"Paid agents dividends of {dividend_amount}!")
        return dividend_amount
    
    def __iter__(self):
        self.curr = 1
        return self
    
    def __len__(self):
        return len(self.prices)
    
    def __next__(self) -> pd.DataFrame:
        if self.curr < len(self.prices.date.unique()):
            
            self.max_date = self.prices.date.sort_values().unique()[self.curr - 1]
            state = self.prices.loc[self.prices.date <= self.max_date]
            self.curr += 1
            return state
        else:
            #  we are on the last date of the prices > exit the iteration
            raise StopIteration
    
    def __str__(self):
        return f"Market [{self.name}] (current market_date: {self.max_date})"

#### Calculation of dividend yield
__On certain date (choose ~~01/02~~ first trading date of year):__
1. Get the last paid dividend amount from prev. year := `dividend_payment`
2. Multiply by the number of times dividends were paid last year (mostly: 4) := `dividend_total`
3. divide `dividend_total` by `current_share_price`

### Strategy Interface

In [12]:
class Strategy:
    def __init__(self, name: str = None):
        self.name = name or type(self).__name__
    
    def weight(self, market_state: pd.DataFrame, agent_portfolio: Dict[str, float]) -> Dict[str, float]:
        pass

__Desired code behavior:__
1. ✅ Check if it's the first trading date of that year
2. ✅ Check if we have a previous year
3. ✅ Get the last-paid dividend of prev. year := `last_paid_div`
4. ✅ Get count of dividends payed of last year per stock := `num_div_paid`
5. ✅ last_paid_div * num_div_paid := `projected_div_amount`
6. ✅Apply weights (how? => all in on highest value stock)

#### Strategy implementation - Dogs of the Stocks

In [13]:
class DogsOfTheStocks(Strategy):
    def __init__(self, top_n_stocks: int=10, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.top_n_stocks = top_n_stocks
    
    def weight(self, market_state: pd.DataFrame, 
               agent_portfolio: Dict[str, float],
              is_recommendation: bool=False) -> Dict[str, float]:
        # make default weight map
        weights = {}
        stocks = set(market_state.symbol.unique())
        for stock in stocks:
            weights[stock] = 0.
        
        current_date = market_state.date.max()
        if current_date.year == market_state.date.min().year:
            # there is no previous year
            return weights
        if not is_recommendation: 
            if current_date != market_state.loc[market_state.date.dt.year == current_date.year].date.min():
                # it's not the first trading day of the current year
                return weights
        
        prev_year = current_date.year - 1
        last_paid_divs = {}
        # all dividends paid last year
        dividends_paid_last_year_mask = (market_state.dividend_amount != 0.) & (market_state.date.dt.year == prev_year)
        divs = market_state.loc[dividends_paid_last_year_mask].sort_values(by="date")
        div_yields = []
        for stock in stocks:
            # for every stock:
            # > get last paid dividend of prev. year
            # > get amount of times dividends were paid last year
            last_paid_divs[stock] = {}
            try:
                # if there were dividends paid last year, grab the last one
                last_paid_divs[stock]["amount"] = divs.loc[divs.symbol == stock].dividend_amount.values[-1]
            except IndexError:
                # there were no dividends paid last year
                last_paid_divs[stock]["amount"] = 0
            last_paid_divs[stock]["times_paid"] = len(divs.loc[divs.symbol == stock])

        for stock, dividend_data in last_paid_divs.items():
            # for every stock: multiply last paid dividend by number of times div
            # were paid last year,
            # and divide by the current stock price
            current_price_mask = (market_state.date == current_date) &\
                (market_state.symbol == stock)
            current_price = market_state.loc[current_price_mask].close.values[0]
            current_stock = (stock, dividend_data["amount"] * dividend_data["times_paid"] / current_price)
            div_yields.append(current_stock)

        # assign weights based on relative share of total yields
        div_yields.sort(key=lambda t: t[1], reverse=True)
        div_yields = div_yields[:self.top_n_stocks]
        total_yield_sum = sum([t[1] for t in div_yields])
        for symbol, div_yield in div_yields:
            weights[symbol] = div_yield / total_yield_sum
        
        # rebalancing
        for symbol in agent_portfolio.keys():
            # Are my current stocks no longer the highest dividend yield stocks? > sell them
            if weights.get(symbol, 0) <= 0:
                weights[symbol] = -1
        
        return weights
    
    def recommend(self, market_state: pd.DataFrame):
        return self.weight(market_state=market_state, agent_portfolio={}, is_recommendation=True)

### Definition of a trading transaction for the agent

In [29]:
Transaction = namedtuple(
    typename="Transaction", 
    field_names=["symbol", "amount", "date", "stock_price", "total_value"]
)

### Definition of a evaluation of an agent's current state

In [28]:
Evaluation = namedtuple(typename="Evaluation", field_names=["timestamp", "score"])

### Evalatuor

In [25]:
class Evaluator:
    def __init__(self):
        self.evaluation_history = [] # List[Tuple(Timestamp, float)]
    
    def evaluate(self, market: Market, date: Timestamp, portfolio: Dict[str, int]) -> float:
        pass

### Agent

In [23]:
class Agent:
    def __init__(self, starting_capital: float, market: Market, strategy: Strategy, name: str):
        # TODO(jonas): implement agent getting new cash every [x interval]
        self.starting_capital = starting_capital
        self.cash = starting_capital
        self.market = market
        self.strategy = strategy
        self.name = name
        self.state = None
        self.portfolio = {}
        self.trading_history = []

    def run_simulation(self):
        for state in market:
            self.state = state
            # agent gets dividends paid out accd. to his portfolio
            self.cash += self.market.pay_dividends(stock_portfolio=self.portfolio)
            weights = self.strategy.weight(market_state=self.state, agent_portfolio=self.portfolio)
            if any([weight != 0 for weight in weights.values()]):
                # any non-0 weights? > evaluate
                for symbol, weight in filter(lambda x: x[1] < 0, weights.items()):
                    # First: look at negative weights := sell recommendations
                    current_stock_price = self.market.get_most_recent_price(symbol=symbol)
                    sell_transaction = self.build_transaction(
                            stock_symbol=symbol,
                            stock_price=current_stock_price,
                            amount=int(weight * self.portfolio.get(symbol)),
                            date=self.market.max_date,
                        )
                    print(f"Making sell transaction: {sell_transaction}")
                    self.sell(sell_transaction)
                amount_to_spend_for_each = (1 / self.strategy.top_n_stocks) * self.cash
                for symbol, weight in filter(lambda x: x[1] > 0, weights.items()):
                    current_stock_price = self.market.get_most_recent_price(symbol=symbol)
                    amount_of_stocks_to_buy = amount_to_spend_for_each // current_stock_price
                    if amount_of_stocks_to_buy == 0:
                        continue
                    purchase_transaction = self.build_transaction(
                        stock_symbol=symbol,
                        stock_price=current_stock_price,
                        amount=amount_of_stocks_to_buy,
                        date=self.market.max_date,
                    )
                    print(f"Making purchase transaction: {purchase_transaction}")
                    self.buy(purchase_transaction)
    
    @staticmethod
    def build_transaction(
            stock_symbol: str,
            stock_price: float,
            amount: int,
            date: Timestamp
    ) -> Transaction:
        total_price = amount * stock_price
        return Transaction(
            symbol=stock_symbol,
            amount=amount,
            date=date,
            stock_price=stock_price,
            total_value=total_price,
        )
    
    def buy(self, transaction: Transaction):
        total_price = transaction.amount * transaction.stock_price
        assert self.cash - total_price >= 0, "Can't spend more than you have!"
        self.cash -= total_price
        self.portfolio[transaction.symbol] = self.portfolio.get(transaction.symbol, 0) + transaction.amount
        self.trading_history.append(transaction)
    
    def sell(self, transaction: Transaction):
        assert transaction.amount < 0, "Sell transactions must have a negative amount!"
        # transaction.amount HAS TO BE NEGATIVE
        total_price = transaction.amount * transaction.stock_price
        self.cash -= total_price
        self.portfolio[transaction.symbol] = self.portfolio.get(transaction.symbol, 0) + transaction.amount
        if self.portfolio.get(transaction.symbol) == 0:
            self.portfolio.pop(transaction.symbol, None)
        self.trading_history.append(transaction)
    
    def print_stats(self):
        spacing = "\n\n=======================================\n\n"
        print("History of transactions in order of occurence:\n")
        for trade in self.trading_history:
            print(trade)
        print(spacing)
        print("Current portfolio:\n")
        pprint.pprint(self.portfolio)
        print(spacing)
        print("Performance verdict:\n")
        print(self.evaluate())
        
    def get_own_total_value(self):
        total_value = self.cash
        for stock, amount in self.portfolio.items():
            total_value += self.market.get_most_recent_price(symbol=stock) * amount
        
        return total_value
    
    def evaluate(self):
        gains_or_losses_percentage = self.get_own_total_value() / self.starting_capital - 1
        percentage_string = "{0:.2%}".format(gains_or_losses_percentage)
        evaluation_sentence = f"Gained {percentage_string}!" if gains_or_losses_percentage > 0 \
            else f"Lost {percentage_string}!"
        return evaluation_sentence
    
    def recommend(self):
        return self.strategy.recommend(market_state=self.state)

### Testing

In [21]:
market = Market(prices=df.loc[df.date.dt.year.isin([2016, 2017, 2018, 2019, 2020])], name="test")
strategy = DogsOfTheStocks(name="DogsOfTheStocks", top_n_stocks=5)
agent = Agent(
    starting_capital=10_000,
    market=market,
    strategy=strategy,
    name="test",
)

agent.run_simulation()

C:\Users\Jonas\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Making purchase transaction: Transaction(symbol='MCD', amount=18.0, date=numpy.datetime64('2017-01-03T00:00:00.000000000'), stock_price=109.6259, total_value=1973.2662)
Making purchase transaction: Transaction(symbol='ESS', amount=9.0, date=numpy.datetime64('2017-01-03T00:00:00.000000000'), stock_price=208.6438, total_value=1877.7942)
Making purchase transaction: Transaction(symbol='YUM', amount=33.0, date=numpy.datetime64('2017-01-03T00:00:00.000000000'), stock_price=59.4908, total_value=1963.1964)
Making purchase transaction: Transaction(symbol='PFE', amount=69.0, date=numpy.datetime64('2017-01-03T00:00:00.000000000'), stock_price=28.9687, total_value=1998.8402999999998)
Making purchase transaction: Transaction(symbol='TPR', amount=63.0, date=numpy.datetime64('2017-01-03T00:00:00.000000000'), stock_price=31.5919, total_value=1990.2897)
Paid agents dividends of 9.9!
Paid agents dividends of 22.080000000000002!
Paid agents dividends of 16.919999999999998!
Paid agents dividends of 21.29

In [16]:
t = Transaction(
    symbol="AAPL",
    amount=10,
    date="2020-01-01",
    stock_price=3_000.00,
    total_value=30_000.00
)

t.amount

10

# To-Do list
1. ✅ Add attributes to Price model, re-pull prices from AV
1. ⬜ Implement 1 preference for demo purposes
1. ✅ Make agent more intelligent (weighting of stocks)
2. ⬜ implement agent getting new cash every x interval
3. ✅ agent gets dividends
4. ✅ dividend amount adapts to amount of stocks agent has
5. ✅ agent buys stuff

In [10]:
import requests
import json
import pprint

In [16]:
payload = {
    "risk_affinity": 1,
    "diversification": 1,
    "placeholder": 1,
    "strategy": "DogsOfTheStocks",
    "starting_capital": 10000
}

url = r"http://localhost:8000/api/sim/start/"

response = requests.post(url=url, json=payload)

In [17]:
pprint.pprint(json.loads(response.json()))

{'current_cash': 3506.7599999999998,
 'current_portfolio': {'AAP': 6.0,
                       'ABBV': 21.0,
                       'ABT': 25.0,
                       'ACN': 10.0,
                       'AES': 126.0,
                       'ATVI': 27.0,
                       'MMM': 7.0},
 'performance': {'current_portfolio_value': 11000.988,
                 'percent_change': 0.10009880000000004,
                 'starting_capital': 10000},
 'transactions': [{'amount': 7.0,
                   'date': '2016-01-04T00:00:00.000000000',
                   'stock_price': 129.119,
                   'symbol': 'MMM',
                   'total_value': 903.833},
                  {'amount': 27.0,
                   'date': '2016-01-04T00:00:00.000000000',
                   'stock_price': 36.371,
                   'symbol': 'ATVI',
                   'total_value': 982.017},
                  {'amount': 25.0,
                   'date': '2016-01-04T00:00:00.000000000',
                   'sto